In [1]:
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip install tweepy
!pip install nltk
!pip install networkx
!pip install textblob
!pip install tensorflow==1.15
!pip install scikit-learn
!pip install tensorflow_hub
!pip install textwrap3
!pip install plotly
!pip install vaderSentiment

In [2]:
# import essential packages
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json
import numpy as np
import os
import itertools
import collections
import tweepy
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob
import pandas as pd
import math
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import json
import warnings
from collections import defaultdict
from sklearn.cluster import KMeans
import textwrap
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly import tools
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
# from utils.text_wapper import remove_url

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
# config data
inputFolder = "./twitter-data-geo-output"

In [5]:
# tweetsText = []
# for file in os.listdir(inputFolder):
#     filePath = "{}/{}".format(inputFolder, file)
#     data = pd.read_json(filePath, orient = 'records', lines = True)

In [6]:
# url_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
# pic_pattern = re.compile('pic\.twitter\.com/.{10}')

In [7]:
## Import the data and create a dict with tweets of a user

tweetsText = []

user_tweets = defaultdict(str)
for file in os.listdir(inputFolder):
    filePath = "{}/{}".format(inputFolder, file)
    with open(filePath, "r") as jsonFile:
        jsf = json.load(jsonFile)
        for j in jsf:
            if j['lang'] == 'en':
                text = j['text']
                text = re.sub(r'http\S+', '', text)
#                 re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
                text.replace('@ ', '@')
                text.replace('# ', '#')
                tweetsText.append(text)
                user_tweets[j['user']['id']] += text
    break

In [8]:
# Get word embedding mod
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
# Get word embeddings
cfg = tf.ConfigProto(allow_soft_placement=True )
cfg.gpu_options.allow_growth = True
with tf.Session(config=cfg) as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(list(user_tweets.values())))
print(message_embeddings.shape)

InternalError: [_Derived_]  Blas GEMM launch failed : a.shape=(2691, 320), b.shape=(320, 320), m=2691, n=320, k=320
	 [[{{node EncoderDNN/DNN/ResidualHidden_0/dense/MatMul}}]]
	 [[StatefulPartitionedCall_1/StatefulPartitionedCall/StatefulPartitionedCall]]
	 [[StatefulPartitionedCall_1/_379]]

In [ ]:
# Bert

# from bert_serving.client import BertClient
# client = BertClient()
# vectors = client.encode(user_tweets_text)
# print(len(vectors))

In [ ]:
#Cluster using word embeddings
X = message_embeddings

kmeans = KMeans(n_clusters=5, random_state=42).fit(message_embeddings)

df = pd.DataFrame(zip(list(user_tweets.keys()), list(user_tweets.values()), kmeans.labels_), columns = ['userid', 'text', 'cluster_text'])

df.head()

# group = defaultdict(list)
# for index, text in enumerate(user_tweets_text):
#     group[kmeans.labels_[index]].append(text)
    
# for cluster_index in group:
#     print(group[cluster_index])
#     print("-----------------------------------------------------------------------------------")


In [ ]:
# Assign Cosine Similality to each Cluster Center
def cos_sim(v1, v2):
    return np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2))

norm = np.array(np.zeros(len(X)*5).reshape(len(X), 5))
for i in range(len(X)):
    for j in range(5):
        norm[i, j] = cos_sim(X[i], kmeans.cluster_centers_[j])
        
grp_df = pd.concat([df,
                    pd.DataFrame(norm, index=df.index).rename(columns={0:'sim0', 1:'sim1', 2:'sim2', 3:'sim3', 4:'sim4'})],
                    axis=1)
# grp_df = pd.DataFrame(norm, message_embeddings).rename(columns={0:'sim0', 1:'sim1', 2:'sim2', 3:'sim3', 4:'sim4'})
# df = pd.DataFrame(list(zip(norm[:], message_embeddings)), columns = ['sim0','sim1','sim2','sim3','sim4', 'text'])

# grp_df = pd.concat([data_president2,
#                     pd.DataFrame(norm, index=data_president2.index).rename(columns={0:'sim0', 1:'sim1', 2:'sim2', 3:'sim3'})],
#                     axis=1)

# Divide tweets data by Cluster
df_0 = grp_df[grp_df['cluster_text']==0].sort_values('sim0', ascending=False)
df_1 = grp_df[grp_df['cluster_text']==1].sort_values('sim1', ascending=False)
df_2 = grp_df[grp_df['cluster_text']==2].sort_values('sim2', ascending=False)
df_3 = grp_df[grp_df['cluster_text']==3].sort_values('sim3', ascending=False)
df_4 = grp_df[grp_df['cluster_text']==4].sort_values('sim3', ascending=False)

grp_df.head()

In [ ]:
def sentiment_analysis(text):
    analysis = TextBlob(text)
    Sentiment = analysis.sentiment[0]
    return Sentiment
 

def subjectivity_analysis(text):
    analysis = TextBlob(text)
    Subjectivity = analysis.sentiment[1]
    return Subjectivity 

df["Sentiment"] = df.text.map(lambda x:sentiment_analysis(x) )
df["Subjectivity"] = df.text.map(lambda x:subjectivity_analysis(x) )

# df[["Sentiment","Subjectivity"]].head()
df.head()

In [ ]:
def typical_tweets(df):
    for i in range(5):
        wrap_list = textwrap.wrap(df.iloc[i,1], 76)
        print('\n'.join(wrap_list),'\n')
        
typical_tweets(df_0)


In [ ]:
typical_tweets(df_1)

In [ ]:
typical_tweets(df_2)


In [ ]:
typical_tweets(df_3)


In [ ]:
typical_tweets(df_4)

In [ ]:
keywords = ['coronavirus', 'COVID', 'virus', 'corona', 'pandemic', 'quarantine']
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def is_relevant(tweet):
    word_tokens = tokenizer.tokenize(tweet)
    word_tokens = [word for word in word_tokens if word not in stop_words]
   # word_tokens = [ps.stem(w) for w in word_tokens]
    result =  any(elem in word_tokens for elem in keywords)
    return result

relevant_tweets = [tweet for tweet in tweetsText if is_relevant(tweet)]

print(len(relevant_tweets))
print(len(tweetsText))



In [ ]:
# for tweet in tweetsText:
#     if is_relevant(tweet):   
#         if word in vocab:
#             vocab[word].add(doc_id)
#         else:
#             vocab[word] = set([doc_id])
#     return vocab      
            
# vocab = parser()
# print(len(vocab))
# tweetsText = [tweet for tweet in tweetsText if keywords in tweet]

In [ ]:
# sentiment_objects = [TextBlob(tweet) for tweet in tweetsText]
# sentiment_objects[0].polarity, sentiment_objects[0]

### Sentiment Analysis Using TextBlob

In [ ]:
#Sentiment analysis in each cluster grouped by word embeddings

for i in range(0,5):
    df_grp = df[df['cluster_text']==i]
    fig, ax = plt.subplots(figsize=(8, 6))
    df_grp[["Sentiment", "Subjectivity"]].hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="purple")
#     plt.title("Sentiments from recent Tweets during the COVID period")
    plt.show()


# group_count = len(group)
# for i in range(0,group_count):
#     sentiment_objects = [TextBlob(tweet) for tweet in group[i]]
#     sentiment_values = [[tweet.sentiment.polarity, tweet.sentiment.subjectivity, str(tweet)] for tweet in sentiment_objects]
# #     sentiment_df = pd.DataFrame(scores)
#     sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "subjectivity", "tweet"])
#     sentiment_df.head()
#     print("\n\n")
#     fig, ax = plt.subplots(figsize=(8, 6))

# # Plot histogram of the polarity values
#     sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
#              ax=ax,
#              color="purple")

#     plt.title("Sentiments from recent Tweets during the COVID period")
#     plt.show()
    
# #     print(sentiment_df[(sentiment_df.polarity > 0.75)])
# #     print(sentiment_df[(sentiment_df.polarity < -0.5)])
    

In [ ]:
# !pip install plotly

# px.scatter(sentiment_df.sample(n=3000,  replace=True), x= "polarity", y = "subjectivity")
fig = px.scatter(df, x=df.Sentiment, y=df.Subjectivity, color=df.cluster_text)
fig.show()

In [ ]:
# print(df.head())
df_kmeans = df[["Sentiment","Subjectivity"]]
df_kmeans.head()
tweets_array = np.array(df_kmeans)
# px.scatter(df, x="Sentiment", y="Subjectivity",color="cluster_sentiment")
n_cluster = 5
pred = KMeans(n_clusters=n_cluster).fit_predict(tweets_array)
df["cluster_sentiment_TextBlob"] = pred
df.head()
px.scatter(df, x="Sentiment", y="Subjectivity",color="cluster_sentiment_TextBlob", hover_data=["text"])


### Sentiment Analysis Using Vader

In [ ]:
# !pip install vaderSentiment
analyser = SentimentIntensityAnalyzer()

scores =[]

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    
for sentence in list(user_tweets.values()):
    score = analyser.polarity_scores(sentence)
    scores.append(score)

# df["Sentiment"] = df.text.map(lambda x:sentiment_analysis(x) )
dataFrame= pd.DataFrame(scores)

print(dataFrame)

print("Overall Sentiment Score for the multiple sentences :- ",dataFrame.mean())